# Using EcoFOCIpy to process raw field data

## Cruise ID
{nw16xx, dy17xx, hx}

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [1]:
import yaml
import glob
import pandas as pd
import os

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2013/CTDcasts/dy1305/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [12]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/btl/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/DY1305 Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'dy1305' #no hyphens
cruise_meta_file = sample_data_dir+'logs/dy1305.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNuts.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd001.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd002.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd003.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd004.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd005.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd006.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd007.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd008.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd009.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd010.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd011.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1305/rawconverted/btl/ctd012.btl
Processing /Users/bell/ecoraid/2013/CTDc

In [13]:
cruise_data['ctd001.btl']

,sal00,sal11,sbeox0mm/kg,sbeox0ps,sbeox1mm/kg,sbeox1ps,sigma-t00,prdm,depsm,t090c,t190c,wetstar,scan,datetime
bottle,,,,,,,,,,,,,,
1.0,32.1848,32.1862,300.721,95.31269,300.264,95.16779,25.467,154.287,152.786,4.7859,4.7882,1.3269,8808.0,2013-04-30 16:47:35


## Load csv Nutrient File

In [15]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),Sil (uM),NO3 (uM),NO2 (uM),NH4 (uM),Temp
0,6,1,1.040,24.0,9.0,0.16,0.93,24.3
1,6,2,1.015,23.0,8.8,0.16,0.84,24.3
2,6,3,0.930,20.7,7.8,0.15,0.60,24.3
3,6,4,0.920,20.5,7.7,0.15,0.65,24.3
4,6,5,0.921,20.6,7.7,0.15,0.58,24.3
...,...,...,...,...,...,...,...,...
580,2,6,1.181,23.7,10.7,0.19,1.56,23.6
581,2,5,1.302,26.7,12.9,0.20,1.26,23.6
582,2,4,1.545,33.7,16.4,0.21,1.09,23.6
583,2,3,1.644,35.0,17.7,0.24,1.41,23.6


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [16]:
# keep_param = ['bottle','prdm']
keep_param = ['bottle','prdm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [22]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [23]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [24]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [25]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil Flag':'SI_WOCE_FLAG',
                            'PO4 Flag':'PO4_WOCE_FLAG',
                            'NO2 Flag':'NO2_WOCE_FLAG', 
                            'NO3 Flag':'NO3_WOCE_FLAG',
                            'NH4 Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')
        os.remove(cruise_name+'c'+cast.zfill(3)+'_nut.nc')

Skipping 001
Skipping 002
Skipping 003
Skipping 004
Skipping 005
Skipping 029
Skipping 030
Skipping 033


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history